In [ ]:
# analytics
import pandas as pd 
import numpy as np
import scipy.stats as stats
import statsmodels.formula.api as smf
#spatial 
import osmnx as ox
import geopandas as gpd
import contextily as cx
# plotting 
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
#settings
import warnings

In [ ]:
model = smf.logit("alerts ~ experience", data=newyork).fit()
print(model.summary())

In [ ]:
model = smf.logit('alerts ~ experience', data=jacksonville).fit()
print(model.summary())